<a href="https://colab.research.google.com/github/pranaydang/Sentiment-Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Rapidsai installation for cuML

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Installing RAPIDS remaining 24.4.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com

        ***********************************************************************
        The pip install of RAPIDS is complete.
        
        Please do not run any further installation from the conda based installation methods, as they may cause issues!
        
        Please ensure that you're pulling from the git repo to remain updated with the latest working install scripts.

        Troubleshooting:
            - If there is an installation failure, please check back on RAPIDSAI owned templates/notebooks to see how to update your personal files. 
            - If an installation failure persists when using the latest script, please make an issue on https://github.com/rapidsai-community/rapidsai-csp-utils
        *****************************************************************

#Change locale to UTF-8 to install Optuna

In [ ]:
import locale

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
print(locale.getpreferredencoding())

UTF-8


In [ ]:
!pip install optuna

#Imports

In [ ]:
import cuml
import cudf
import cupy
from transformers import BertTokenizer, BertModel
import pandas as pd
import torch
import numpy as np
import xgboost as xgb
import lightgbm as lgb
import optuna
from cuml.decomposition import PCA
from cuml.preprocessing import StandardScaler, LabelEncoder
from cuml.metrics.accuracy import accuracy_score
from cuml.ensemble import RandomForestClassifier
from cuml.svm import SVC
from cuml.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold

#Load weights and set BERT model to evaluation mode

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained('bert-base-cased')
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

#Read data + Rename headers

In [ ]:
csv_file = 'all-data.csv'

with open(csv_file, encoding='ISO-8859-1') as file:
    data = pd.read_csv(file)

df = pd.DataFrame(data)

In [ ]:
df.rename(columns={'neutral': 'sentiment', 'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .': 'news'}, inplace=True)

#Generate embeddings

In [ ]:
def process_batch(batch_df):
    inputs = tokenizer(list(batch_df['news']), return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        return outputs.last_hidden_state[:, 0, :].numpy()

embeddings = []

for start in range(0, len(df), 500):
    end = start + 500
    batch_df = df.iloc[start:end]
    batch_embeddings = process_batch(batch_df)
    embeddings.extend(batch_embeddings)

df['embedding'] = embeddings

In [ ]:
embeddings_df = pd.DataFrame(embeddings)

embeddings_df.columns = [f'embedding_{i}' for i in range(embeddings_df.shape[1])]

df = pd.concat([df, embeddings_df], axis=1)
print(df.head())

  sentiment                                               news  \
0   neutral  Technopolis plans to develop in stages an area...   
1  negative  The international electronic industry company ...   
2  positive  With the new production plant the company woul...   
3  positive  According to the company 's updated strategy f...   
4  positive  FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...   

                                           embedding  embedding_0  \
0  [0.30768058, -0.101167955, -0.2680171, -0.2473...     0.307681   
1  [0.4039932, -0.05932038, -0.2536986, -0.206923...     0.403993   
2  [0.21252042, 0.034812164, -0.103083834, -0.116...     0.212520   
3  [0.3080871, 0.04362351, -0.1663971, -0.2106388...     0.308087   
4  [0.36219856, 0.038855486, -0.07439936, -0.2339...     0.362199   

   embedding_1  embedding_2  embedding_3  embedding_4  embedding_5  \
0    -0.101168    -0.268017    -0.247340    -0.353130     0.045741   
1    -0.059320    -0.253699    -0.206923    -0.3

#Making dataframes, splitting into test and train, and making a cuDF DataFrame

In [ ]:
x = df.iloc[:, 3:]
y = df['sentiment']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
x_train_cu = cudf.DataFrame(x_train)
x_test = cudf.DataFrame(x_test)
y_train = cudf.Series(y_train)
y_test = cudf.Series(y_test)

#PCA

In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

n_components = 300
pca = PCA(n_components=n_components)
x_train_pca = pca.fit_transform(x_train_scaled)
x_test_pca = pca.transform(x_test_scaled)

explained_varience_ratio = np.sum(pca.explained_variance_ratio_)
print(explained_varience_ratio)

0.9561912


#Trial with Random Forest, XGBoost, LightGBM and SVC

In [ ]:
models = {"Random Forest": RandomForestClassifier(),"SVC": SVC(), "XGBoost": xgb.XGBClassifier(use_label_encode=False, eval_metric='mlogloss'), "LightGBM": lgb.LGBMClassifier()}

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

x_train_pca_numpy = x_train_pca.to_numpy()
x_test_pca_numpy = x_test_pca.to_numpy()
y_train_encoded_numpy = y_train_encoded.to_numpy()
y_test_encoded_numpy = y_test_encoded.to_numpy()

results = []

for name, model in models.items():
    print(f"Training {name}...")

    if name == "XGBoost":
        dtrain = xgb.DMatrix(x_train_pca_numpy, label=y_train_encoded_numpy)
        dtest = xgb.DMatrix(x_test_pca_numpy, label=y_test_encoded_numpy)
        model.fit(x_train_pca_numpy, y_train_encoded_numpy)
        y_pred = model.predict(x_test_pca_numpy)
    elif name == "LightGBM":
        model.fit(x_train_pca_numpy, y_train_encoded_numpy)
        y_pred = model.predict(x_test_pca_numpy)
    else:
        model.fit(x_train_pca, y_train_encoded)
        y_pred = model.predict(x_test_pca)

    accuracy = accuracy_score(y_test_encoded, y_pred)
    results.append([name, accuracy])

results_df = cudf.DataFrame(results, columns=["Model", "Accuracy"])
print(results_df)

Training Random Forest...
Training SVC...
[W] [19:13:00.682565] Warning: could not fill working set, found only 936 elements
[W] [19:13:00.689208] Warning: could not fill working set, found only 949 elements
[W] [19:13:00.691795] Warning: could not fill working set, found only 937 elements
[W] [19:13:00.693523] Warning: could not fill working set, found only 895 elements
[W] [19:13:00.695406] Warning: could not fill working set, found only 924 elements
[W] [19:13:00.696893] Warning: could not fill working set, found only 895 elements
[W] [19:13:00.717757] Warning: could not fill working set, found only 936 elements
[W] [19:13:00.720531] Warning: could not fill working set, found only 981 elements
[W] [19:13:00.722564] Warning: could not fill working set, found only 903 elements
[W] [19:13:00.724696] Warning: could not fill working set, found only 914 elements
[W] [19:13:00.726848] Warning: could not fill working set, found only 892 elements
[W] [19:13:00.727924] Warning: could not fill

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [19:13:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encode" } are not used.

  warnings.warn(smsg, UserWarning)


Training LightGBM...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004006 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 3391, number of used features: 300
[LightGBM] [Info] Start training from score -2.079147
[LightGBM] [Info] Start training from score -0.512597
[LightGBM] [Info] Start training from score -1.287265
           Model  Accuracy
0  Random Forest  0.665062
1            SVC  0.800550
2        XGBoost  0.768226
3       LightGBM  0.762036


#Hyperparameter optimization on SVC

In [ ]:
def objective(trial):
    C = trial.suggest_loguniform('C', 1e-4, 1e2)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'sigmoid', 'poly'])
    gamma = 'scale'

    if kernel in ['rbf', 'sigmoid', 'poly']:
        gamma = trial.suggest_loguniform('gamma', 1e-4, 1e1)

    if kernel == 'poly':
        degree = trial.suggest_int('degree', 2, 5)
    else:
        degree = 3

    svc = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree, max_iter=1000, random_state=42)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    f1_scores = []
    accuracy_train = []
    accuracy_val = []
    for train_index, test_index in skf.split(x_train_pca_numpy, y_train_encoded_numpy):
        x_train, x_val = x_train_pca_numpy[train_index], x_train_pca_numpy[test_index]
        y_train, y_val = y_train_encoded_numpy[train_index], y_train_encoded_numpy[test_index]

        x_train_cu = cudf.DataFrame.from_records(x_train)
        x_val_cu = cudf.DataFrame.from_records(x_val)
        y_train_cu = cudf.Series(y_train)
        y_val_cu = cudf.Series(y_val)

        svc.fit(x_train_cu, y_train_cu)
        y_pred = svc.predict(x_val_cu)
        y_pred_test = svc.predict(x_train_cu)

        f1 = f1_score(y_val, y_pred.to_numpy(), average='macro')
        accuracy_train.append(accuracy_score(y_train, y_pred_test.to_numpy()))
        accuracy_val.append(accuracy_score(y_val, y_pred.to_numpy()))
        f1_scores.append(f1)

        if (sum(accuracy_train) / len(accuracy_train)) - (sum(accuracy_val) / len(accuracy_val)) > 0.05:
            return -100

    return sum(f1_scores) / len(f1_scores)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200, n_jobs=-1)

print(study.best_trial)

[I 2024-09-01 19:57:42,059] A new study created in memory with name: no-name-57b548df-84b7-4245-aa10-81acba7a690d
<ipython-input-32-860d05a1bd12>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e2)
<ipython-input-32-860d05a1bd12>:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-4, 1e1)
<ipython-input-32-860d05a1bd12>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e2)
<ipython-input-3

[W] [19:57:44.802174] Warning: could not fill working set, found only 851 elements
[W] [19:57:44.828199] Warning: could not fill working set, found only 768 elements
[W] [19:57:44.880098] Warning: could not fill working set, found only 980 elements
[W] [19:57:44.951060] Warning: could not fill working set, found only 768 elements
[W] [19:57:45.025057] Warning: could not fill working set, found only 997 elements
[W] [19:57:45.088960] Warning: could not fill working set, found only 768 elements
[W] [19:57:45.146090] Warning: could not fill working set, found only 997 elements
[W] [19:57:45.180190] Warning: could not fill working set, found only 768 elements
[W] [19:57:45.191397] Warning: could not fill working set, found only 997 elements
[W] [19:57:45.216495] Warning: could not fill working set, found only 768 elements
[W] [19:57:45.315879] Warning: could not fill working set, found only 851 elements
[W] [19:57:45.339246] Warning: could not fill working set, found only 768 elements
[W] 

[I 2024-09-01 19:57:55,638] Trial 1 finished with value: -100.0 and parameters: {'C': 0.12448176851553457, 'kernel': 'poly', 'gamma': 0.007346933589911769, 'degree': 3}. Best is trial 1 with value: -100.0.
<ipython-input-32-860d05a1bd12>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e2)
<ipython-input-32-860d05a1bd12>:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-4, 1e1)
[I 2024-09-01 19:57:55,814] Trial 5 finished with value: -100.0 and parameters: {'C': 0.0041783375406366065, 'kernel': 'linear'}. Best is trial 1 with value: -100.0.
[I 2024-09-01 19:57:55,837] Trial 2 finished 

[W] [19:57:58.522147] Warning: could not fill working set, found only 851 elements
[W] [19:57:58.544515] Warning: could not fill working set, found only 829 elements
[W] [19:57:58.547442] Warning: could not fill working set, found only 851 elements
[W] [19:57:58.560480] Warning: could not fill working set, found only 962 elements
[W] [19:57:58.560661] Warning: could not fill working set, found only 851 elements
[W] [19:57:58.568820] Warning: could not fill working set, found only 805 elements
[W] [19:57:58.574067] Warning: could not fill working set, found only 801 elements
[W] [19:57:58.595935] Warning: could not fill working set, found only 768 elements
[W] [19:57:58.655513] Warning: could not fill working set, found only 851 elements
[W] [19:57:58.658261] Warning: could not fill working set, found only 980 elements
[W] [19:57:58.732988] Warning: could not fill working set, found only 851 elements
[W] [19:57:58.734889] Warning: could not fill working set, found only 769 elements
[W] 

[I 2024-09-01 19:58:04,437] Trial 11 finished with value: -100.0 and parameters: {'C': 0.016353689884551045, 'kernel': 'linear'}. Best is trial 1 with value: -100.0.
<ipython-input-32-860d05a1bd12>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e2)
<ipython-input-32-860d05a1bd12>:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-4, 1e1)
[I 2024-09-01 19:58:04,536] Trial 9 finished with value: -100.0 and parameters: {'C': 0.6796387405529032, 'kernel': 'linear'}. Best is trial 1 with value: -100.0.
<ipython-input-32-860d05a1bd12>:9: FutureWarning: suggest_loguniform has been deprecated

...

[W] [20:08:58.216273] Warning: could not fill working set, found only 851 elements
[W] [20:08:58.219512] Warning: could not fill working set, found only 824 elements
[W] [20:08:58.222140] Warning: could not fill working set, found only 959 elements
[W] [20:08:58.225452] Warning: could not fill working set, found only 866 elements
[W] [20:08:58.231391] Warning: could not fill working set, found only 964 elements
[W] [20:08:58.240392] Warning: could not fill working set, found only 867 elements
[W] [20:08:58.256866] Warning: could not fill working set, found only 851 elements
[W] [20:08:58.260136] Warning: could not fill working set, found only 849 elements
[W] [20:08:58.261596] Warning: could not fill working set, found only 963 elements
[W] [20:08:58.263682] Warning: could not fill working set, found only 863 elements
[W] [20:08:58.266560] Warning: could not fill working set, found only 969 elements
[W] [20:08:58.270306] Warning: could not fill working set, found only 863 elements
[W] 

[I 2024-09-01 20:08:58,920] Trial 198 finished with value: 0.7495788518608658 and parameters: {'C': 4.684100185565723, 'kernel': 'sigmoid', 'gamma': 0.0006011441960378821}. Best is trial 186 with value: 0.7507823276455255.


[W] [20:08:59.002330] Warning: could not fill working set, found only 852 elements
[W] [20:08:59.007264] Warning: could not fill working set, found only 808 elements
[W] [20:08:59.009638] Warning: could not fill working set, found only 969 elements
[W] [20:08:59.012749] Warning: could not fill working set, found only 862 elements
[W] [20:08:59.015282] Warning: could not fill working set, found only 977 elements
[W] [20:08:59.033455] Warning: could not fill working set, found only 863 elements
[W] [20:08:59.045326] Warning: could not fill working set, found only 852 elements
[W] [20:08:59.048532] Warning: could not fill working set, found only 851 elements
[W] [20:08:59.049896] Warning: could not fill working set, found only 971 elements
[W] [20:08:59.052661] Warning: could not fill working set, found only 856 elements
[W] [20:08:59.062449] Warning: could not fill working set, found only 978 elements
[W] [20:08:59.069680] Warning: could not fill working set, found only 857 elements


[I 2024-09-01 20:08:59,239] Trial 199 finished with value: 0.7420157102865053 and parameters: {'C': 9.44998729568036, 'kernel': 'sigmoid', 'gamma': 0.000650760905003981}. Best is trial 186 with value: 0.7507823276455255.


FrozenTrial(number=186, state=TrialState.COMPLETE, values=[0.7507823276455255], datetime_start=datetime.datetime(2024, 9, 1, 20, 8, 10, 181441), datetime_complete=datetime.datetime(2024, 9, 1, 20, 8, 33, 345944), params={'C': 5.082402177490603, 'kernel': 'sigmoid', 'gamma': 0.0005981823503646617}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'C': FloatDistribution(high=100.0, log=True, low=0.0001, step=None), 'kernel': CategoricalDistribution(choices=('linear', 'rbf', 'sigmoid', 'poly')), 'gamma': FloatDistribution(high=10.0, log=True, low=0.0001, step=None)}, trial_id=186, value=None)


#Best model

In [ ]:
params={'C': 8.382543800300043, 'kernel': 'sigmoid', 'gamma': 0.00015236099901369673, 'max_iter':1000, 'random_state':42}
svc = SVC(**params)
svc.fit(x_train_pca, y_train_encoded)
y_pred_test = svc.predict(x_test_pca)
y_pred_train = svc.predict(x_train_pca)
accuracy_test = accuracy_score(y_test_encoded, y_pred_test)
accuracy_train = accuracy_score(y_train_encoded, y_pred_train)
print(f"Test Accuracy: {accuracy_test}")
print(f"Train Accuracy: {accuracy_train}")

[W] [20:10:14.324892] Warning: could not fill working set, found only 936 elements
[W] [20:10:14.330487] Warning: could not fill working set, found only 894 elements
[W] [20:10:14.335128] Warning: could not fill working set, found only 959 elements
[W] [20:10:14.339332] Warning: could not fill working set, found only 956 elements
[W] [20:10:14.344363] Warning: could not fill working set, found only 952 elements
[W] [20:10:14.347374] Warning: could not fill working set, found only 957 elements
[W] [20:10:14.360768] Warning: could not fill working set, found only 936 elements
[W] [20:10:14.366102] Warning: could not fill working set, found only 919 elements
[W] [20:10:14.369976] Warning: could not fill working set, found only 950 elements
[W] [20:10:14.374042] Warning: could not fill working set, found only 963 elements
[W] [20:10:14.378990] Warning: could not fill working set, found only 948 elements
[W] [20:10:14.382395] Warning: could not fill working set, found only 961 elements
Test